In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from scipy import stats as sps
from scipy.interpolate import interp1d
from IPython.display import clear_output

In [0]:
!git clone https://github.com/yenlow/twitterpandemicindex.git

Cloning into 'twitterpandemicindex'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 424 (delta 0), reused 0 (delta 0), pack-reused 420
Receiving objects: 100% (424/424), 70.69 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (204/204), done.


In [0]:
cd twitterpandemicindex/data 


[Errno 2] No such file or directory: 'twitterpandemicindex/data'
/root/twitterpandemicindex/data


In [0]:
!unzip maindf.zip

Archive:  maindf.zip
replace maindf.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: maindf.csv              


Maindf is loaded, dates are parsed, and index is declared as a combination of state, county, and date

In [0]:
rt_df = pd.read_csv("maindf.csv", 
                    usecols = ["state-county", "Date", "Confirmed"], 
                    parse_dates=["Date"],
                    index_col = ["state-county", "Date"],
                    squeeze = True).sort_index()

Gamma (a constant used in the likelihood function) is declared as 1/7 according to CDC and WHO findings

In [0]:
R_T_MAX = 5
r_t_range = np.linspace(0, R_T_MAX, R_T_MAX*100+1)
GAMMA = 1/7

A cutoff of at least 25 cases is chosen. A smoothed version of each time series is created

In [0]:
def prepare_cases(cases, cutoff=25):
    new_cases = cases.diff()
    smoothed = new_cases.rolling(7,
        win_type='gaussian',
        min_periods=1,
        center=True).mean(std=2).round()
    
    idx_start = np.searchsorted(smoothed, cutoff)
    
    smoothed = smoothed.iloc[idx_start:]
    original = new_cases.loc[smoothed.index]
    
    return original, smoothed

The next function applies a bayesian model for finding Rt in the absence of an optimized sigma, it provides .15 as a default

In [0]:
def get_posteriors(sr, sigma=0.15):

    lam = sr[:-1].values * np.exp(GAMMA * (r_t_range[:, None] - 1))
    
    likelihoods = pd.DataFrame(
        data = sps.poisson.pmf(sr[1:].values, lam),
        index = r_t_range,
        columns = sr.index[1:])
    process_matrix = sps.norm(loc=r_t_range,
                              scale=sigma
                             ).pdf(r_t_range[:, None]) 
    process_matrix /= process_matrix.sum(axis=0)
    prior0 = np.ones_like(r_t_range)/len(r_t_range)
    prior0 /= prior0.sum()

    posteriors = pd.DataFrame(
        index=r_t_range,
        columns=sr.index,
        data={sr.index[0]: prior0}
    )
    log_likelihood = 0.0
    for previous_day, current_day in zip(sr.index[:-1], sr.index[1:]):

        current_prior = process_matrix @ posteriors[previous_day]
        numerator = likelihoods[current_day] * current_prior
        denominator = np.sum(numerator)
        posteriors[current_day] = numerator/denominator
        log_likelihood += np.log(denominator)
    
    return posteriors, log_likelihood

The next process finds an optimal sigma for each state-county combination, and performs the Rt calculation for every sigma

In [0]:
sigmas = np.linspace(1/20, 1, 20)

results = {}
for state_name, cases in rt_df.groupby(level='state-county'):
    print(state_name)
    new, smoothed = prepare_cases(cases, cutoff=5)
    
    if len(smoothed) == 0:
        continue
    
    result = {}
    
    result['posteriors'] = []
    result['log_likelihoods'] = []
    for sigma in sigmas:
        posteriors, log_likelihood = get_posteriors(smoothed, sigma=sigma)
        result['posteriors'].append(posteriors)
        result['log_likelihoods'].append(log_likelihood)
    results[state_name] = result
    clear_output(wait=True)

print('Done.')

Wyoming-Uinta
Wyoming-Unassigned
Wyoming-Washakie
Wyoming-Weston
Done.


The optimal value of sigma is identified 

In [0]:
total_log_likelihoods = np.zeros_like(sigmas)

for state_name, result in results.items():
    total_log_likelihoods += result['log_likelihoods']

max_likelihood_index = total_log_likelihoods.argmax()
sigma = sigmas[max_likelihood_index]

The final result for each state is then extracted from the posteriors' matrix

In [0]:
final_results = None

for state_name, result in results.items():
    print(state_name)
    posteriors = result['posteriors'][max_likelihood_index]
    most_likely = posteriors.idxmax().rename('ML')
    result = pd.concat([most_likely], axis=1)
    if final_results is None:
        final_results = result
    else:
        final_results = pd.concat([final_results, result])
    clear_output(wait=True)

print('Done.')

Done.


In [0]:
final_results.to_csv("final_results.csv")

In [0]:
final_results = pd.read_csv("final_results.csv")

In [0]:
state = []

county = []

for row in range(0, len(final_results.index)):
    state.append(final_results.loc[row, "state-county"].split("-")[0])
    county.append(final_results.loc[row, "state-county"].split("-")[1])
    
final_results.loc[:, "state"] = state
final_results.loc[:, "county"] = county

In [0]:
final_results.to_csv("final_results.csv")

"""
REFERENCES

Systrom, K. (2020) Estimating COVID-19's Rt in Real-Time. Retrieved from https://github.com/k-sys/covid-19/blob/master/Realtime%20R0.ipynb on 04/20/2020 and its implementation by Stephanie Dominguez.


"""

"\nREFERENCES\n\nSystrom, K. (2020) Estimating COVID-19's Rt in Real-Time. Retrieved from https://github.com/k-sys/covid-19/blob/master/Realtime%20R0.ipynb on 04/20/2020\n\n"

In [0]:
from google.colab import files
files.download('final_results.csv') 